In [20]:
import cx_Oracle
import os

In [21]:
os.environ["NLS_LANG"] = "POLISH_POLAND.AL32UTF8"
connection = cx_Oracle.connect("C##PPAWLUK", "#Lex2018", "144.21.89.60:1521/ORCL.591263988.oraclecloud.internal")

In [22]:
cur = connection.cursor()

In [23]:
cur.execute('DELETE FROM "C##PPAWLUK"."TEMAT_DOKUMENTU"')
cur.execute('commit')

In [24]:
dokumentsName = cur.execute('select distinct nazwa_dokumentu from dokument')

In [25]:
names = []

In [26]:
for dokument in dokumentsName:
    names.append(dokument[0])

In [27]:
dokumentsText = []

In [28]:
j = 0
for name in names:
    dokumentsText.append({'name': name, 'text': ''})
    text = cur.execute('select tekst from dokument where nazwa_dokumentu = \'{}\''.format(name))
    for t in text:
        dokumentsText[j]['text'] = dokumentsText[j]['text'] + t[0]
    j = j + 1

In [29]:
allTekst = ''

In [30]:
for z in dokumentsText:
    allTekst = allTekst + z['text']

In [31]:
allTekst = allTekst.split(' ')
allTekst.remove('')
allTekst = list(set(allTekst))

In [32]:
matrix = []

In [33]:
for word in allTekst:
    for dokument in names:
        matrix.append({"name":dokument,"word": word,'frequency':0})

In [34]:
for dokument in dokumentsText:
    text = dokument['text']
    text = text.split(' ')
    text.remove('')
    for m in matrix:
        if(m['name'] == dokument['name']):
            m['frequency'] = text.count(m['word'])

In [35]:
def themaDoc(matrix,nameDoc):
    tmpMatrix = []
    for m in matrix:
        if(m['name'] == nameDoc and m['frequency'] > 1):
            tmpMatrix.append(m)
    sortMatrix = sorted(tmpMatrix, key=lambda x: x['frequency'],reverse=True)
    size = 10
    if( len(sortMatrix) < size):
        size = len(sortMatrix)
    tematy = ''
    for i in range(0,size,1):
         tematy = tematy + ' ' + sortMatrix[i]['word']
    cur.execute(
        'INSERT INTO "C##PPAWLUK"."TEMAT_DOKUMENTU" (NAZWA_DOKUMENTU, TEMATY) VALUES( \'{}\', \'{}\')'.format(
            nameDoc, tematy))

In [36]:
for name in names:
    themaDoc(matrix,name)

In [37]:
cur.execute('commit')

In [38]:
cur.close()
connection.close()